In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from prettytable import PrettyTable


In [4]:
def avg_diff(values):
    # return the average pairwise different between all the pairs in the list
    pairs = [(a, b) for idx, a in enumerate(values) for b in values[idx + 1:]]
    difference = list(map(lambda x: abs(x[0] - x[1]), pairs))
    return sum(difference)/len(difference)


In [20]:
score_dir = '../bold/result/score'
path_to_dir = os.path.relpath(score_dir)
# get the json files in the directory
csv_files = [pos_csv for pos_csv in os.listdir(path_to_dir) if pos_csv.endswith('.csv')]

df = pd.concat([pd.read_csv(os.path.join(path_to_dir, file))
               for file in csv_files])


,model,group,subgroup,metric,score
0,gpt2,race,Asian_Americans,regard-positive,0.4000
1,gpt2,race,Asian_Americans,regard-negative,0.2000
2,gpt2,race,Asian_Americans,regard-neutral,0.2000
3,gpt2,race,Asian_Americans,regard-other,0.2000
4,gpt2,race,Asian_Americans,toxicity-ratio,0.0000
5,gpt2,race,African_Americans,regard-positive,1.0000
6,gpt2,race,African_Americans,regard-negative,0.0000
7,gpt2,race,African_Americans,regard-neutral,0.0000
8,gpt2,race,African_Americans,regard-other,0.0000
9,gpt2,race,African_Americans,toxicity-ratio,0.0000


In [21]:
regardTable = PrettyTable(
    ["Domain", "Model", "Positive", "Negative", " Nuetral"])
toxicityTable = PrettyTable(["Domain", "Model", "Ratio"])


In [22]:
scores = []
groups = df['group'].unique().tolist()

for group in groups:
    # group = 'political_ideology'
    data = df[df['group'] == group]
    model_name = data[data['metric'] ==
                      'regard-positive']['model'].values.tolist()
    subgroup = data[data['metric'] ==
                    'regard-positive']['subgroup'].values.tolist()
    labels = [model_name[i] + '\n' + subgroup[i]
              for i in range(len(model_name))]

    model_name = data[data['metric'] ==
                      'regard-positive']['model'].unique().tolist()
    subgroup = data[data['metric'] ==
                    'regard-positive']['subgroup'].unique().tolist()

    positive_regards = np.array(
        data[data['metric'] == 'regard-positive']['score'].values.tolist())
    negative_regards = np.array(
        data[data['metric'] == 'regard-negative']['score'].values.tolist())
    neutral_regards = np.array(
        data[data['metric'] == 'regard-neutral']['score'].values.tolist())
    toxicity = np.array(
        data[data['metric'] == 'toxicity-ratio']['score'].values.tolist())

    n_subgroups = len(subgroup)
    for i in range(len(model_name)):
        start_ind, end_ind = n_subgroups * i, n_subgroups * (i+1)
        positive = positive_regards[start_ind:end_ind]
        negative = negative_regards[start_ind:end_ind]
        neutral = neutral_regards[start_ind:end_ind]
        toxic = toxicity[start_ind:end_ind]

        regardTable.add_row([group, model_name[i], round(avg_diff(positive), 4), round(
            avg_diff(negative), 4), round(avg_diff(neutral), 4)])
        toxicityTable.add_row(
            [group, model_name[i], round(avg_diff(toxic), 4)])

        scores.append({'model': model_name[i], 'group': group,  'positive': round(avg_diff(positive), 4),
                       'negative': round(avg_diff(negative), 4), 'neutral': round(avg_diff(neutral), 4), 'toxicity_ratio': round(avg_diff(toxic), 4)})


In [23]:
print(regardTable)


+--------------------+-------+----------+----------+----------+
|       Domain       | Model | Positive | Negative |  Nuetral |
+--------------------+-------+----------+----------+----------+
|        race        |  gpt2 |  0.4397  |  0.1762  |  0.3079  |
| religious_ideology |  gpt2 |  0.1676  |  0.1383  |  0.1148  |
+--------------------+-------+----------+----------+----------+


In [24]:
print(toxicityTable)


+--------------------+-------+--------+
|       Domain       | Model | Ratio  |
+--------------------+-------+--------+
|        race        |  gpt2 |  0.0   |
| religious_ideology |  gpt2 | 0.0646 |
+--------------------+-------+--------+


In [25]:
#save to csv
df = pd.DataFrame.from_dict(scores)
df.to_csv (r'relative_regard_scores.csv', index = False, header=True)